# Read in LGA list and clean up for use in google api loops

In [99]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup as bs
# Google API Key
from config import gkey

#file path to saved csv dataframe
file = 'Raw_Data/Vic-LGA-List.csv'

In [16]:
#bring csv file in as pd dataframe
council_city_df = pd.read_csv(file, encoding="ISO-8859-1")
council_city_df

,ï»¿COUNCIL,CEO NAME,DEAR CEO,CEO TITLE,MAYOR NAME,DEAR MAYOR,MAYOR TITLE,POSTAL,POSTAL SUBURB,PCODE,DX,ADDRESS,SUBURB/ TOWN,CODE,TEL,FAX,EMAIL,WWW SITE
0,Alpine Shire Council,Mr Charlie Bird,Mr Bird,Chief Executive Officer,Cr Peter Roper,Cr Roper,Mayor,PO Box 139,BRIGHT,3741,,Great Alpine Road,BRIGHT,3741,5755 0555,5755 1811,info@alpineshire.vic.gov.au,www.alpineshire.vic.gov.au
1,Ararat Rural City Council,Dr Tim Harrison,Dr Harrison,Chief Executive Officer,Cr Jo Armstrong,Cr Armstrong,Mayor,PO Box 246,ARARAT,3377,,59 Vincent Street,ARARAT,3377,5355 0200,5352 1695,council@ararat.vic.gov.au,www.ararat.vic.gov.au
2,Ballarat City Council,Ms Janet Dore,Ms Dore,Chief Executive Officer,Cr Ben Taylor,Cr Taylor,Mayor,PO Box 655,BALLARAT,3353,,25 Armstrong Street South,BALLARAT,3350,5320 5500,5333 4061,info@ballarat.vic.gov.au,www.ballarat.vic.gov.au
3,Banyule City Council,Ms Allison Beckwith,Ms Beckwith,Chief Executive Officer,Cr Alison Champion,Cr Champion,Mayor,PO Box 94,GREENSBOROUGH,3088,DX 97904 Ivanhoe,1 Flintoff Street,GREENSBOROUGH,3088,9490 4222,9499 9475,enquiries@banyule.vic.gov.au,www.banyule.vic.gov.au
4,Bass Coast Shire Council,Ms Ali Wastie,Ms Wastie,Chief Executive Officer,Cr Brett Tessari,Cr Tessari,Mayor,PO Box 118,WONTHAGGI,3995,DX 34903 Wonthaggi,76 McBride Avenue,WONTHAGGI,3995,1300 BCOAST (226278) or 5671 2211,5671 2222,basscoast@basscoast.vic.gov.au,www.basscoast.vic.gov.au
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Wodonga City Council,Mr Mark Dixon,Mr Dixon,Chief Executive Officer,Cr Anna Speedie,Cr Speedie,Mayor,PO Box 923,WODONGA,3689,,104 Hovell Street,WODONGA,3690,02 6022 9300,02 6022 9322,info@wodonga.vic.gov.au,www.wodonga.vic.gov.au
75,Wyndham City Council,Ms Kelly Grigsby,Ms Grigsby,Chief Executive Officer,Cr Josh Gilligan,Cr Gilligan,Mayor,PO Box 197,WERRIBEE,3030,,45 Princes Highway,WERRIBEE,3030,9742 0777,9741 6237,mail@wyndham.vic.gov.au,www.wyndham.vic.gov.au
76,Yarra City Council,Ms Vijaya Vaidyanath,Ms Vaidyanath,Chief Executive Officer,Cr Misha Coleman,Cr Coleman,Mayor,PO Box 168,RICHMOND,3121,DX 30205 Richmond,333 Bridge Road,RICHMOND,3121,9205 5555,,info@yarracity.vic.gov.au,www.yarracity.vic.gov.au
77,Yarra Ranges Shire Council,Ms Tammi Rose,Ms Rose,Chief Executive Officer,Cr Richard Higgins,Cr Higgins,Mayor,PO Box 105,LILYDALE,3140,DX 34051 Lilydale,Anderson Street,LILYDALE,3140,1300 368 333,9735 4249,mail@yarraranges.vic.gov.au,www.yarraranges.vic.gov.au


In [17]:
print('The number of Local Government Areas (councils) in Vic is: '+str(len(council_city_df)))

The number of Local Government Areas (councils) in Vic is: 79


In [18]:
# get list of column headings for cleaning up df
council_city_df.columns.unique()

Index(['ï»¿COUNCIL', 'CEO NAME', 'DEAR CEO', 'CEO TITLE', 'MAYOR NAME',
       'DEAR MAYOR', 'MAYOR TITLE', 'POSTAL', 'POSTAL SUBURB', 'PCODE', 'DX',
       'ADDRESS', 'SUBURB/ TOWN', 'CODE', 'TEL', 'FAX', 'EMAIL', 'WWW SITE'],
      dtype='object')

In [19]:
# clean council_city_df to have just the columns wanted
council_city_df=council_city_df.drop(['CEO NAME', 'DEAR CEO', 'CEO TITLE', 
                                      'MAYOR NAME','DEAR MAYOR', 'MAYOR TITLE', 
                                      'POSTAL', 'POSTAL SUBURB', 'PCODE', 'DX',
                                      'ADDRESS','TEL', 'FAX', 'EMAIL'], axis=1)
# re-name council column
council_city_df = council_city_df.rename(columns={"ï»¿COUNCIL": "COUNCIL"})
council_city_df

,COUNCIL,SUBURB/ TOWN,CODE,WWW SITE
0,Alpine Shire Council,BRIGHT,3741,www.alpineshire.vic.gov.au
1,Ararat Rural City Council,ARARAT,3377,www.ararat.vic.gov.au
2,Ballarat City Council,BALLARAT,3350,www.ballarat.vic.gov.au
3,Banyule City Council,GREENSBOROUGH,3088,www.banyule.vic.gov.au
4,Bass Coast Shire Council,WONTHAGGI,3995,www.basscoast.vic.gov.au
...,...,...,...,...
74,Wodonga City Council,WODONGA,3690,www.wodonga.vic.gov.au
75,Wyndham City Council,WERRIBEE,3030,www.wyndham.vic.gov.au
76,Yarra City Council,RICHMOND,3121,www.yarracity.vic.gov.au
77,Yarra Ranges Shire Council,LILYDALE,3140,www.yarraranges.vic.gov.au


In [ ]:
# remove melb from df
cities_df_xmelb = cities_df.drop([1977])
cities_df_xmelb

In [ ]:
print(len(cities_df_xmelb))

In [20]:
# make city list
cities = council_city_df["SUBURB/ TOWN"].tolist()
print(cities)

['BRIGHT', 'ARARAT', 'BALLARAT', 'GREENSBOROUGH', 'WONTHAGGI', 'DROUIN', 'SANDRINGHAM', 'BENALLA', 'CAMBERWELL', 'SUNSHINE', 'WYCHEPROOF', 'ECHUCA', 'OFFICER', 'NARRE WARREN', 'MARYBOROUGH', 'COLAC', 'CAMPERDOWN', 'PRESTON', 'BAIRNSDALE', 'FRANKSTON', 'KERANG', 'CAULFIELD SOUTH', 'PORTLAND', 'BANNOCKBURN', 'BENDIGO', 'DANDENONG', 'GEELONG', 'SHEPPARTON', 'DAYLESFORD', 'NHILL', 'ALTONA', 'HORSHAM', 'BROADMEADOWS', 'BEECHWORTH', 'CHELTENHAM', 'WANTIRNA SOUTH', 'MORWELL', 'WEDDERBURN', 'KYNETON', 'DONCASTER', 'MANSFIELD', 'FOOTSCRAY', 'RINGWOOD', 'MELBOURNE', 'MELTON', 'MILDURA', 'BROADFORD', 'COBRAM', 'GLEN WAVERLEY', 'MOONEE PONDS', 'BALLAN', 'COBURG', 'ROSEBUD', 'CASTLEMAINE', 'PORT FAIRY', 'ALEXANDRA', 'GREENSBOROUGH', 'STAWELL', 'ST KILDA', 'BEAUFORT', 'QUEENSCLIFF', 'LEONGATHA', 'HAMILTON', 'MALVERN', 'EUROA', 'TORQUAY', 'SWAN HILL', 'TALLANGATTA', 'WANGARATTA', 'WARRNAMBOOL', 'SALE', 'EDENHOPE', 'NUNAWADING', 'SOUTH MORANG', 'WODONGA', 'WERRIBEE', 'RICHMOND', 'LILYDALE', 'WARRACKNA

In [21]:
cities_list = np.array(cities)
print(cities_list)

['BRIGHT' 'ARARAT' 'BALLARAT' 'GREENSBOROUGH' 'WONTHAGGI' 'DROUIN'
 'SANDRINGHAM' 'BENALLA' 'CAMBERWELL' 'SUNSHINE' 'WYCHEPROOF' 'ECHUCA'
 'OFFICER' 'NARRE WARREN' 'MARYBOROUGH' 'COLAC' 'CAMPERDOWN' 'PRESTON'
 'BAIRNSDALE' 'FRANKSTON' 'KERANG' 'CAULFIELD SOUTH' 'PORTLAND'
 'BANNOCKBURN' 'BENDIGO' 'DANDENONG' 'GEELONG' 'SHEPPARTON' 'DAYLESFORD'
 'NHILL' 'ALTONA' 'HORSHAM' 'BROADMEADOWS' 'BEECHWORTH' 'CHELTENHAM'
 'WANTIRNA SOUTH' 'MORWELL' 'WEDDERBURN' 'KYNETON' 'DONCASTER' 'MANSFIELD'
 'FOOTSCRAY' 'RINGWOOD' 'MELBOURNE' 'MELTON' 'MILDURA' 'BROADFORD'
 'COBRAM' 'GLEN WAVERLEY' 'MOONEE PONDS' 'BALLAN' 'COBURG' 'ROSEBUD'
 'CASTLEMAINE' 'PORT FAIRY' 'ALEXANDRA' 'GREENSBOROUGH' 'STAWELL'
 'ST KILDA' 'BEAUFORT' 'QUEENSCLIFF' 'LEONGATHA' 'HAMILTON' 'MALVERN'
 'EUROA' 'TORQUAY' 'SWAN HILL' 'TALLANGATTA' 'WANGARATTA' 'WARRNAMBOOL'
 'SALE' 'EDENHOPE' 'NUNAWADING' 'SOUTH MORANG' 'WODONGA' 'WERRIBEE'
 'RICHMOND' 'LILYDALE' 'WARRACKNABEAL']


# Get Tourist Attractions using loop through cities and calling google api

In [28]:
# use google api to search things to do for each city

#lists to append to
name_ta = []
address_ta = []
rating_ta = []
city_ta = []
city_searched = []
for city in cities_list:
    citi = city
    city_ta.append(citi)
    my_phrase = f"tourist attraction establishment in {city} VIC, AU"
    target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
    "query": my_phrase,
    "key": gkey
    }
    response = requests.get(target_url, params)
    to_dos = response.json()
    counter = 0
    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
    print("----------------------")
    #print(response.url)
    #print(json.dumps(to_dos, indent=4, sort_keys=True))
    for do in to_dos["results"]:
        counter += 1
        try:
            print("-------------------")
            print(f'{citi}')
            city_searched.append(citi)
            called = do["name"]
            locate = do["formatted_address"]
            rated = do["rating"]
            if(called, locate and rated):
                print("-------------------")
                name_ta.append(called)
                print(called)
                address_ta.append(locate)
                print(locate)
                rating_ta.append(rated)
                print(rated)
        except:
            print('not found')           
    
    if counter == 20:
        time.sleep(1.01)
        counter = 0


----------------------
-------------------
BRIGHT
-------------------
Huggins Lookout Point
Huggins Rd, Bright VIC 3741, Australia
4.4
-------------------
BRIGHT
-------------------
Apex Lookout
Bright VIC 3741, Australia
4.4
-------------------
BRIGHT
-------------------
Howitt Park
7 Riverside Ave, Bright VIC 3741, Australia
4.6
-------------------
BRIGHT
-------------------
Tower Hill Lookout
Bright VIC 3741, Australia
4.4
-------------------
BRIGHT
-------------------
Bright Centenary Park
Canyon Walk, Bright VIC 3741, Australia
4.7
-------------------
BRIGHT
-------------------
Chinese Bridge
Wandiligong VIC 3744, Australia
4.3
-------------------
BRIGHT
-------------------
Canyon Walk
8 Star Rd, Bright VIC 3741, Australia
4.6
-------------------
BRIGHT
-------------------
Chinese Pavilion
Wandiligong VIC 3744, Australia
4
-------------------
BRIGHT
-------------------
Apex Park
79 Gavan St, Bright VIC 3741, Australia
4.7
-------------------
BRIGHT
-------------------
Wandi Woods


----------------------
-------------------
WONTHAGGI
-------------------
State Coal Mine
State Coal Mine, Garden St, Wonthaggi VIC 3995, Australia
4.7
-------------------
WONTHAGGI
-------------------
Tank Hill Reserve
Wonthaggi VIC 3995, Australia
4.5
-------------------
WONTHAGGI
-------------------
WILLIAMSON'S BEACH
Lower Powlett Rd, Wonthaggi VIC 3995, Australia
4.6
-------------------
WONTHAGGI
-------------------
Baxters Wetland
165 Campbell St, Wonthaggi VIC 3995, Australia
5
-------------------
WONTHAGGI
-------------------
Wonthaggi Railway Museum
Wonthaggi VIC 3995, Australia
5
-------------------
WONTHAGGI
-------------------
Guide Park
Wonthaggi VIC 3995, Australia
4.3
-------------------
WONTHAGGI
-------------------
Wonthaggi Cenotaph
Wonthaggi VIC 3995, Australia
5
-------------------
WONTHAGGI
-------------------
Wonthaggi showground
109 Korumburra Rd, Wonthaggi VIC 3995, Australia
4.2
-------------------
WONTHAGGI
-------------------
Wonthaggi Wetlands Conservation Pa

----------------------
-------------------
CAMBERWELL
-------------------
Read Gardens
24 Stanhope Grove, Camberwell VIC 3124, Australia
4.5
-------------------
CAMBERWELL
-------------------
Riversdale Park
Camberwell VIC 3124, Australia
4.3
-------------------
CAMBERWELL
-------------------
Through Road Reserve
Through Rd, Camberwell VIC 3124, Australia
4.4
-------------------
CAMBERWELL
-------------------
Frog Hollow Reserve
25 Fordham Ave, Camberwell VIC 3124, Australia
4.2
-------------------
CAMBERWELL
-------------------
Fordham Gardens
Camberwell VIC 3124, Australia
4.6
-------------------
CAMBERWELL
-------------------
Camberwell Sunday Market
Market Pl, Camberwell VIC 3124, Australia
4.3
-------------------
CAMBERWELL
-------------------
Powell Avenue Reserve
410 Camberwell Rd, Camberwell VIC 3124, Australia
3
-------------------
CAMBERWELL
-------------------
Canterbury Gardens
Canterbury VIC 3126, Australia
4.5
-------------------
CAMBERWELL
-------------------
Lynden Park

----------------------
-------------------
OFFICER
-------------------
Kangaroo Flat Botanical Gardens
Kangaroo Flat VIC 3555, Australia
3.8
-------------------
OFFICER
-------------------
Poppet Head Lookout
30 Gaol Rd, Bendigo VIC 3550, Australia
4.6
-------------------
OFFICER
-------------------
Central Deborah Gold Mine
76 Violet St, Bendigo VIC 3550, Australia
4.6
-------------------
OFFICER
-------------------
Discovery Science & Technology Centre
7 Railway Pl, Bendigo VIC 3550, Australia
4.3
-------------------
OFFICER
-------------------
Bendigo Tourism
Pall Mall, Bendigo VIC 3550, Australia
0
-------------------
OFFICER
-------------------
Rosalind Park
30 Gaol Rd, Bendigo VIC 3550, Australia
4.5
-------------------
OFFICER
-------------------
Bendigo Tramways
1 Tramways Ave, Bendigo VIC 3550, Australia
4.6
-------------------
OFFICER
-------------------
Victoria Hill Mining Reserve
Ironbark VIC 3550, Australia
4.6
-------------------
OFFICER
-------------------
Alexandra Fou

----------------------
-------------------
CAMPERDOWN
-------------------
Mt Leura lookout
Mt Leura Rd, Camperdown VIC 3260, Australia
4.6
-------------------
CAMPERDOWN
-------------------
The Courthouse Camperdown
179 Manifold St, Camperdown VIC 3260, Australia
5
-------------------
CAMPERDOWN
-------------------
Camperdown Buggy Museum
26 Ower St, Camperdown VIC 3260, Australia
0
-------------------
CAMPERDOWN
-------------------
Dr. Daniel Curdie
Camperdown VIC 3260, Australia
4.5
-------------------
CAMPERDOWN
-------------------
Curdie Reserve
13-15 Curdie St, Camperdown VIC 3260, Australia
5
-------------------
CAMPERDOWN
-------------------
Wombeech Puyuun Reconciliation Park
2B Walker St, Camperdown VIC 3260, Australia
4
-------------------
CAMPERDOWN
-------------------
Mockridge Park
10 Thornton St, Camperdown VIC 3260, Australia
4
-------------------
CAMPERDOWN
-------------------
Lakes and Craters Holiday Park
220 Park Rd, Camperdown VIC 3260, Australia
4.1
---------------

...

----------------------
-------------------
QUEENSCLIFF
-------------------
Queenscliff Harbour
1 Harbour St, Queenscliff VIC 3225, Australia
4.5
-------------------
QUEENSCLIFF
-------------------
Fort Queenscliff Museum
Cnr King &, Gellibrand St, Queenscliff VIC 3225, Australia
4.4
-------------------
QUEENSCLIFF
-------------------
Queenscliff High Light
Queenscliff VIC 3225, Australia
4.3
-------------------
QUEENSCLIFF
-------------------
The Bellarine Railway - Queenscliff Station
20 Symonds St, Queenscliff VIC 3225, Australia
4.5
-------------------
QUEENSCLIFF
-------------------
Queenscliffe Maritime Museum
Maritime Museum, 2 Wharf St, Queenscliff VIC 3225, Australia
4.6
-------------------
QUEENSCLIFF
-------------------
Seaview Gallery
86 Hesse St, Queenscliff VIC 3225, Australia
4.8
-------------------
QUEENSCLIFF
-------------------
Queenscliffe Historical Museum
49 Hesse St, Queenscliff VIC 3225, Australia
4.4
-------------------
QUEENSCLIFF
-------------------
Princess Pa

----------------------
-------------------
EUROA
-------------------
Euroa Swaggy Tree
&, Spencer Street, Slee St, Euroa VIC 3666, Australia
5
-------------------
EUROA
-------------------
Euroa Arboretum
Euroa Main Road, 76 Hume Fwy, Euroa VIC 3666, Australia
4.1
-------------------
EUROA
-------------------
Euroa Apex Park
66/68 Kirkland Ave, Euroa VIC 3666, Australia
4.3
-------------------
EUROA
-------------------
Euroa Miniature Railway
11 Turnbull St, Euroa VIC 3666, Australia
4.5
-------------------
EUROA
-------------------
Euroa Farmers Arms Hotel Museum
25 Kirkland Ave, Euroa VIC 3666, Australia
4.5
-------------------
EUROA
-------------------
Seven Creeks Park
46 Kirkland Ave W, Euroa VIC 3666, Australia
4.5
-------------------
EUROA
-------------------
Euroa Historic Courthouse
99 Binney St, Euroa VIC 3666, Australia
4.7
-------------------
EUROA
-------------------
Forlonge Memorial
137/619 Forlonge Memorial Rd, Euroa VIC 3666, Australia
4.7
-------------------
EUROA
---

----------------------
-------------------
WANGARATTA
-------------------
Merriwa Park
Ryley Street, Wangaratta VIC 3677, Australia
4.5
-------------------
WANGARATTA
-------------------
Wangaratta Common Nature Conservation Reserve
Tone Rd, Wangaratta VIC 3677, Australia
4
-------------------
WANGARATTA
-------------------
Kaluna Park
Wangaratta VIC 3677, Australia
5
-------------------
WANGARATTA
-------------------
Northern Beaches
Wangaratta VIC 3677, Australia
4.4
-------------------
WANGARATTA
-------------------
Mullinmur Wetlands - Wangaratta
2 Cambridge Dr, Wangaratta VIC 3677, Australia
4
-------------------
WANGARATTA
-------------------
Ovens Billabong Bushland Reserve
Wangaratta VIC 3677, Australia
4.3
-------------------
WANGARATTA
-------------------
Wangaratta Rose Garden
Wangaratta VIC 3677, Australia
5
-------------------
WANGARATTA
-------------------
Bullawah Cultural Trail
Wangaratta VIC 3677, Australia
4.1
-------------------
WANGARATTA
-------------------
Wangara

----------------------
-------------------
NUNAWADING
-------------------
Tunstall Park
Nunawading VIC 3131, Australia
4.3
-------------------
NUNAWADING
-------------------
Lindsay Avenue Linear Reserve
43B Springvale Rd, Nunawading VIC 3131, Australia
3.5
-------------------
NUNAWADING
-------------------
Jubilee Road Reserve
39 Jubilee St, Nunawading VIC 3131, Australia
4.7
-------------------
NUNAWADING
-------------------
Nangnak Garden
Nunawading VIC 3131, Australia
4.5
-------------------
NUNAWADING
-------------------
Nicoll Park
Nunawading VIC 3131, Australia
4.4
-------------------
NUNAWADING
-------------------
Mullum Mullum Park
Donvale VIC 3111, Australia
4.5
-------------------
NUNAWADING
-------------------
Koonung Creek Trail
Koonung Creek Trail, Nunawading VIC 3131, Australia
5
-------------------
NUNAWADING
-------------------
Charles Rooks Reserve
Nunawading VIC 3131, Australia
3.9
-------------------
NUNAWADING
-------------------
Box Hill to Ringwood shared use pat

----------------------
-------------------
RICHMOND
-------------------
Melbourne Cricket Ground
Brunton Ave, Richmond VIC 3002, Australia
4.7
-------------------
RICHMOND
-------------------
Burnley Park
Bellevue St, Richmond VIC 3121, Australia
4.4
-------------------
RICHMOND
-------------------
Barkly Gardens
Coppin Street and, Mary St, Richmond VIC 3121, Australia
4.5
-------------------
RICHMOND
-------------------
Richmond Catholic Parish
326 Church St, Richmond VIC 3121, Australia
4.4
-------------------
RICHMOND
-------------------
Citizens Park
Richmond VIC 3121, Australia
4.3
-------------------
RICHMOND
-------------------
Richmond Football Club Museum
Jack Dyer Stand - Richmond Football Club, Punt Rd, East Melbourne VIC 3002, Australia
4.3
-------------------
RICHMOND
-------------------
Jack Dyer Statue
Unnamed Road, East Melbourne VIC 3002, Australia
5
-------------------
RICHMOND
-------------------
Kings Domain
Alexandra Ave, Melbourne VIC 3004, Australia
4.6
---------

# Create and clean df from tourist attraction data gathered 

In [29]:
print("Total returned Tourist Attractions: "+ str(len(name_ta)))

Total returned Tourist Attractions: 1580


In [30]:
#create a df from tourise attractions
tourist_site_df = pd.DataFrame(name_ta)
tourist_site_df["tourist_site_address"]= address_ta
tourist_site_df["tourist_site_rating"]= rating_ta
tourist_site_df["tourist_site_city"]= city_searched
# Rename column 0 tourist_sites_df
tourist_site_df = tourist_site_df.rename(columns={0: "tourist_site_name"})
tourist_site_df

,tourist_site_name,tourist_site_address,tourist_site_rating,tourist_site_city
0,Huggins Lookout Point,"Huggins Rd, Bright VIC 3741, Australia",4.4,BRIGHT
1,Apex Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
2,Howitt Park,"7 Riverside Ave, Bright VIC 3741, Australia",4.6,BRIGHT
3,Tower Hill Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
4,Bright Centenary Park,"Canyon Walk, Bright VIC 3741, Australia",4.7,BRIGHT
...,...,...,...,...
1575,Warracknabeal Primary School,"5 Werrigar St, Warracknabeal VIC 3393, Australia",5.0,WARRACKNABEAL
1576,Werrigar Roadhouse & Motel,"213 Henty Hwy, Warracknabeal VIC 3393, Australia",4.3,WARRACKNABEAL
1577,Warracknabeal Secondary College,"1 Tregear St, Warracknabeal VIC 3393, Australia",0.0,WARRACKNABEAL
1578,Warracknabeal Early Learning Centre,"11 Werrigar St, Warracknabeal VIC 3393, Australia",0.0,WARRACKNABEAL


In [45]:
#drop duplicates in tourist_sites_df
tourist_site_xdbl_df=tourist_site_df.drop_duplicates(subset=['tourist_site_name', 'tourist_site_city'],keep='last')
tourist_site_xdbl_df

,tourist_site_name,tourist_site_address,tourist_site_rating,tourist_site_city
0,Huggins Lookout Point,"Huggins Rd, Bright VIC 3741, Australia",4.4,BRIGHT
1,Apex Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
2,Howitt Park,"7 Riverside Ave, Bright VIC 3741, Australia",4.6,BRIGHT
3,Tower Hill Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
4,Bright Centenary Park,"Canyon Walk, Bright VIC 3741, Australia",4.7,BRIGHT
...,...,...,...,...
1575,Warracknabeal Primary School,"5 Werrigar St, Warracknabeal VIC 3393, Australia",5.0,WARRACKNABEAL
1576,Werrigar Roadhouse & Motel,"213 Henty Hwy, Warracknabeal VIC 3393, Australia",4.3,WARRACKNABEAL
1577,Warracknabeal Secondary College,"1 Tregear St, Warracknabeal VIC 3393, Australia",0.0,WARRACKNABEAL
1578,Warracknabeal Early Learning Centre,"11 Werrigar St, Warracknabeal VIC 3393, Australia",0.0,WARRACKNABEAL


In [37]:
# save csv of tourist sites
tourist_site_xdbl_df.to_csv("Cleaned_Data/tourist_sites.csv", encoding="utf-8", index=False)

# Get Resturants and cafes using loop through cities and calling google api

In [40]:
# Loop through each city to search for resturants and cafes 
#lists to append to
name_eat = []
address_eat = []
rating_eat =[]
city_search_eat = []
for city in cities_list:
    citi = city
    my_phrase = f"resturant cafe establishment in {city} VIC, AU"
    target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
    "query": my_phrase,
    "key": gkey
    }
    response = requests.get(target_url, params)
    foodies = response.json()
    counter = 0
    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
    print("----------------------")
    #print(response.url)
    #print(json.dumps(to_dos, indent=4, sort_keys=True))
    for food in foodies["results"]:
        counter += 1
        try:
            print("-------------------")
            print(f'{citi}')
            city_search_eat.append(citi)
            city_eat = food
            print("-------------------")
            print(f'{counter}')
            print("-------------------")
            called = food["name"]
            locate = food["formatted_address"]
            rated = food["rating"]
            if(city_eat, called, locate and rated):
                print("-------------------")
                name_eat.append(called)
                print(called)
                address_eat.append(locate)
                print(locate)
                rating_eat.append(rated)
                print(rated)
        except:
            print('not found')            
    
    if counter == 20:
        time.sleep(1.01)
        counter = 0



----------------------
-------------------
BRIGHT
-------------------
1
-------------------
-------------------
Suganya's Thai Restaurant
2/11 Ireland St, Bright VIC 3741, Australia
4.2
-------------------
BRIGHT
-------------------
2
-------------------
-------------------
The Riverdeck Kitchen
16 Howitt Ln, Bright VIC 3741, Australia
4.2
-------------------
BRIGHT
-------------------
3
-------------------
-------------------
Bright Chinese Restaurant
108 Gavan St, Bright VIC 3741, Australia
3.5
-------------------
BRIGHT
-------------------
4
-------------------
-------------------
Elm Dining
98 Gavan St, Bright VIC 3741, Australia
4.6
-------------------
BRIGHT
-------------------
5
-------------------
-------------------
Alpine Hotel Bright
7-9 Anderson St, Bright VIC 3741, Australia
4
-------------------
BRIGHT
-------------------
6
-------------------
-------------------
Wild Thyme Cafe Bright
3 Ireland St, Bright VIC 3741, Australia
4.6
-------------------
BRIGHT
---------------

----------------------
-------------------
GREENSBOROUGH
-------------------
1
-------------------
-------------------
Peppercorn Thai Restaurant
81 Grimshaw St, Greensborough VIC 3088, Australia
4.1
-------------------
GREENSBOROUGH
-------------------
2
-------------------
-------------------
Urban Grooves
99 Grimshaw St, Greensborough VIC 3088, Australia
4.4
-------------------
GREENSBOROUGH
-------------------
3
-------------------
-------------------
Infuse Greensborough
91 Grimshaw St, Greensborough VIC 3088, Australia
4.6
-------------------
GREENSBOROUGH
-------------------
4
-------------------
-------------------
Oriental Jade Restaurant in Greensborough
84 Main St, Greensborough VIC 3088, Australia
3.6
-------------------
GREENSBOROUGH
-------------------
5
-------------------
-------------------
Clay Oven Pizza Restaurant
19 Greenhill Rd, Greensborough VIC 3088, Australia
4.5
-------------------
GREENSBOROUGH
-------------------
6
-------------------
-------------------
Shi

----------------------
-------------------
SANDRINGHAM
-------------------
1
-------------------
-------------------
Restaurant Sandringham Hotel
Beach Rd &, Bay Rd, Sandringham VIC 3191, Australia
0
-------------------
SANDRINGHAM
-------------------
2
-------------------
-------------------
Harbour View Restaurant
Jetty Rd, Sandringham VIC 3191, Australia
4.3
-------------------
SANDRINGHAM
-------------------
3
-------------------
-------------------
Hobsons
19-21 Melrose St, Sandringham VIC 3191, Australia
4.2
-------------------
SANDRINGHAM
-------------------
4
-------------------
-------------------
Sandringham Yacht Club
Jetty Rd, Sandringham VIC 3191, Australia
4.5
-------------------
SANDRINGHAM
-------------------
5
-------------------
-------------------
Le Phan Vietnamese Restaurant
5 Melrose St, Sandringham VIC 3191, Australia
4.3
-------------------
SANDRINGHAM
-------------------
6
-------------------
-------------------
Sook Sabai Thai Restaurant
84 Station St, Sandrin

----------------------
-------------------
SUNSHINE
-------------------
1
-------------------
-------------------
Café Sunshine and SalamaTea Restaurant
21 Dickson St, Sunshine VIC 3020, Australia
4.8
-------------------
SUNSHINE
-------------------
2
-------------------
-------------------
Sunshine Social - BBQ
64 Glengala Rd, Sunshine West VIC 3020, Australia
4.3
-------------------
SUNSHINE
-------------------
3
-------------------
-------------------
Afghan Shaheen Restaurant
231 Hampshire Rd, Sunshine VIC 3020, Australia
4.3
-------------------
SUNSHINE
-------------------
4
-------------------
-------------------
Jinhui Restaurant
491 Ballarat Rd, Sunshine VIC 3020, Australia
4.3
-------------------
SUNSHINE
-------------------
5
-------------------
-------------------
Cafe Mambo
8/80 Harvester Rd, Sunshine VIC 3020, Australia
4.2
-------------------
SUNSHINE
-------------------
6
-------------------
-------------------
Thuan An Restaurant
253 Hampshire Rd, Sunshine VIC 3020, Aus

----------------------
-------------------
NARRE WARREN
-------------------
1
-------------------
-------------------
Pind Baluchi Indian & vegan Restaurant
2 Rebound Ct, Narre Warren VIC 3805, Australia
4.2
-------------------
NARRE WARREN
-------------------
2
-------------------
-------------------
Metioric Café
352 Overland Dr, Narre Warren VIC 3805, Australia
4
-------------------
NARRE WARREN
-------------------
3
-------------------
-------------------
Frankie & Co Cafe
shop 3/10 Victor Cres, Narre Warren VIC 3805, Australia
4.4
-------------------
NARRE WARREN
-------------------
4
-------------------
-------------------
Riks Mahal Indian Restaurant
2/2-8 Victor Cres, Narre Warren VIC 3805, Australia
4.1
-------------------
NARRE WARREN
-------------------
5
-------------------
-------------------
Switch Lifestyle Fountain Gate
Westfield Fountain Gate 1125, 352 Princes Hwy, Narre Warren VIC 3805, Australia
3.5
-------------------
NARRE WARREN
-------------------
6
-------------

...

----------------------
-------------------
MALVERN
-------------------
1
-------------------
-------------------
Zikrayat Lebanese Restaurant
229 Glenferrie Rd, Malvern VIC 3144, Australia
4.4
-------------------
MALVERN
-------------------
2
-------------------
-------------------
Giorgios Restaurant
1235 High St, Malvern VIC 3144, Australia
4.3
-------------------
MALVERN
-------------------
3
-------------------
-------------------
Sugo Restaurant
105 Wattletree Rd, Malvern VIC 3144, Australia
4.2
-------------------
MALVERN
-------------------
4
-------------------
-------------------
Bellezza Restaurant
1413 Malvern Rd, Malvern VIC 3144, Australia
4.5
-------------------
MALVERN
-------------------
5
-------------------
-------------------
Ayame Japanese Restaurant
237 Glenferrie Rd, Malvern VIC 3144, Australia
4.1
-------------------
MALVERN
-------------------
6
-------------------
-------------------
Caffe La Via
252-254 Glenferrie Rd, Malvern VIC 3144, Australia
4
------------

----------------------
-------------------
SWAN HILL
-------------------
1
-------------------
-------------------
Spoons Riverside
125 Monash Dr, Swan Hill VIC 3585, Australia
4.4
-------------------
SWAN HILL
-------------------
2
-------------------
-------------------
Swan Hill Club
5/17 McCallum St, Swan Hill VIC 3585, Australia
4.1
-------------------
SWAN HILL
-------------------
3
-------------------
-------------------
Quo-Vadis Restaurant & Pizza Parlour
255-259 Campbell St, Swan Hill VIC 3585, Australia
4
-------------------
SWAN HILL
-------------------
4
-------------------
-------------------
Colombo's Cafe & Pasta Bar
32 McCrae St, Swan Hill VIC 3585, Australia
4.8
-------------------
SWAN HILL
-------------------
5
-------------------
-------------------
Niko Niko Swan hill Japanese & Korean Restaurant
174 Campbell St, Swan Hill VIC 3585, Australia
4.5
-------------------
SWAN HILL
-------------------
6
-------------------
-------------------
Peppercorn on Campbell
396 

----------------------
-------------------
SALE
-------------------
1
-------------------
-------------------
Gippsland Palace Asian Restaurant
58 MacArthur St, Sale VIC 3850, Australia
4
-------------------
SALE
-------------------
2
-------------------
-------------------
redd catt
192 Raymond St, Sale VIC 3850, Australia
4.6
-------------------
SALE
-------------------
3
-------------------
-------------------
Raglan Pizza Restaurant
221 Raglan St, Sale VIC 3850, Australia
4.3
-------------------
SALE
-------------------
4
-------------------
-------------------
El-Sombrero Mexican Restaurant
168 Raymond St, Sale VIC 3850, Australia
3.8
-------------------
SALE
-------------------
5
-------------------
-------------------
La Porchetta Sale
69 MacArthur St, Sale VIC 3850, Australia
3.9
-------------------
SALE
-------------------
6
-------------------
-------------------
Café No 8
37-39 Cunninghame St, Sale VIC 3850, Australia
3.3
-------------------
SALE
-------------------
7
------

----------------------
-------------------
WODONGA
-------------------
1
-------------------
-------------------
The Secret Cup
86 High St, Wodonga VIC 3690, Australia
4.6
-------------------
WODONGA
-------------------
2
-------------------
-------------------
Phaboonchai Thai Restaurant
Shop 17/202A Beechworth Rd, Wodonga VIC 3690, Australia
4.3
-------------------
WODONGA
-------------------
3
-------------------
-------------------
Elgin's of Albury Wodonga
51-53 Elgin Blvd, Wodonga VIC 3690, Australia
4
-------------------
WODONGA
-------------------
4
-------------------
-------------------
The Cube Cafe (Wodonga)
118 Hovell St, Wodonga VIC 3690, Australia
3.7
-------------------
WODONGA
-------------------
5
-------------------
-------------------
Miss Amelie
46 Elgin Blvd, Wodonga VIC 3690, Australia
4.7
-------------------
WODONGA
-------------------
6
-------------------
-------------------
Steakpit Restaurant
51/53 Elgin Blvd, Wodonga VIC 3690, Australia
4.1
----------------

----------------------
-------------------
LILYDALE
-------------------
1
-------------------
-------------------
OTree Restaurant & Receptions
2 Victoria Rd, Lilydale VIC 3140, Australia
4.6
-------------------
LILYDALE
-------------------
2
-------------------
-------------------
The Valley Restaurant @ The International Lilydale
471 Maroondah Hwy, Lilydale VIC 3140, Australia
3.9
-------------------
LILYDALE
-------------------
3
-------------------
-------------------
Hutch & Co Cafe
251 Main St, Lilydale VIC 3140, Australia
4.5
-------------------
LILYDALE
-------------------
4
-------------------
-------------------
Bom Gosto Restaurant & Cafe
11 Castella St, Lilydale VIC 3140, Australia
4.2
-------------------
LILYDALE
-------------------
5
-------------------
-------------------
Hok's Cafe & Restaurant
236 Main St, Lilydale VIC 3140, Australia
4.7
-------------------
LILYDALE
-------------------
6
-------------------
-------------------
Tiantian Kitchen Chinese Restaurant(Lilyd

# Create and clean df from eatery data gathered 

In [82]:
print("Total returned Eateries: "+ str(len(name_eat)))

Total returned Eateries: 1449


In [83]:
#create a df from eateries
eateries_df = pd.DataFrame(name_eat)
eateries_df["eatery_address"]= address_eat
eateries_df["eatery_rating"]= rating_eat
eateries_df["eatery_city"]= city_search_eat
# Rename column 0 eateries_df
eateries_df = eateries_df.rename(columns={0: "eatery_name"})
eateries_df

,eatery_name,eatery_address,eatery_rating,eatery_city
0,Suganya's Thai Restaurant,"2/11 Ireland St, Bright VIC 3741, Australia",4.2,BRIGHT
1,The Riverdeck Kitchen,"16 Howitt Ln, Bright VIC 3741, Australia",4.2,BRIGHT
2,Bright Chinese Restaurant,"108 Gavan St, Bright VIC 3741, Australia",3.5,BRIGHT
3,Elm Dining,"98 Gavan St, Bright VIC 3741, Australia",4.6,BRIGHT
4,Alpine Hotel Bright,"7-9 Anderson St, Bright VIC 3741, Australia",4.0,BRIGHT
...,...,...,...,...
1444,Cafe Pharmacino,"106-108 Scott St, Warracknabeal VIC 3393, Aust...",0.0,WARRACKNABEAL
1445,Cafe Peppercorn,"122 Scott St, Warracknabeal VIC 3393, Australia",4.4,WARRACKNABEAL
1446,Down Town Takeaway,"77 Scott St, Warracknabeal VIC 3393, Australia",4.0,WARRACKNABEAL
1447,MKS Warracknabeal Kebab & Pizza,"167 Scott St, Warracknabeal VIC 3393, Australia",4.4,WARRACKNABEAL


In [44]:
#drop duplicates in eateries_df
eatery_xdbl_df=eateries_df.drop_duplicates(subset=['eatery_name', 'eatery_city'],keep='last')
eatery_xdbl_df

,eatery_name,eatery_address,eatery_rating,eatery_city
0,Suganya's Thai Restaurant,"2/11 Ireland St, Bright VIC 3741, Australia",4.2,BRIGHT
1,The Riverdeck Kitchen,"16 Howitt Ln, Bright VIC 3741, Australia",4.2,BRIGHT
2,Bright Chinese Restaurant,"108 Gavan St, Bright VIC 3741, Australia",3.5,BRIGHT
3,Elm Dining,"98 Gavan St, Bright VIC 3741, Australia",4.6,BRIGHT
4,Alpine Hotel Bright,"7-9 Anderson St, Bright VIC 3741, Australia",4.0,BRIGHT
...,...,...,...,...
1444,Cafe Pharmacino,"106-108 Scott St, Warracknabeal VIC 3393, Aust...",0.0,WARRACKNABEAL
1445,Cafe Peppercorn,"122 Scott St, Warracknabeal VIC 3393, Australia",4.4,WARRACKNABEAL
1446,Down Town Takeaway,"77 Scott St, Warracknabeal VIC 3393, Australia",4.0,WARRACKNABEAL
1447,MKS Warracknabeal Kebab & Pizza,"167 Scott St, Warracknabeal VIC 3393, Australia",4.4,WARRACKNABEAL


In [46]:
# save csv of eateries
eatery_xdbl_df.to_csv("Cleaned_Data/eateries.csv", encoding="utf-8", index=False)

# Get accommodation using loop through cities and calling google api

In [73]:
# Loop through each city to search for accommodation
#lists to append to
name_stay = []
address_stay = []
rating_stay =[]
city_search_stay = []

for city in cities_list:
    citi = city
    my_phrase = f"hotel motel accommodation near {city} VIC, AU"
    target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
    "query": my_phrase,
    "key": gkey
    }
    response = requests.get(target_url, params)
    stays = response.json()
    counter = 0
    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
    print("----------------------")
    #print(response.url)
    #print(json.dumps(to_dos, indent=4, sort_keys=True))
    for stay in stays["results"]:
        counter += 1
        try:
            print("-------------------")
            print(f'{citi}')
            city_search_stay.append(citi)
            city_eat = stay
            print("-------------------")
            print(f'{counter}')
            print("-------------------")
            called = stay["name"]
            locate = stay["formatted_address"]
            rated = stay["rating"]
            if(city_eat, called, locate and rated):
                print("-------------------")
                name_stay.append(called)
                print(called)
                address_stay.append(locate)
                print(locate)
                rating_stay.append(rated)
                print(rated)
        except:
            print('not found')           
    
    if counter == 20:
        time.sleep(1.01)
        counter = 0

----------------------
-------------------
BRIGHT
-------------------
1
-------------------
-------------------
John Bright Motor Inn
10-12 Wood St, Bright VIC 3741, Australia
4.3
-------------------
BRIGHT
-------------------
2
-------------------
-------------------
Riverbank Park Motel
69 Gavan St, Bright VIC 3741, Australia
4.1
-------------------
BRIGHT
-------------------
3
-------------------
-------------------
High Country Motor Inn
13-17 Great Alpine Rd, Bright VIC 3741, Australia
3.9
-------------------
BRIGHT
-------------------
4
-------------------
-------------------
Bright Avenue Motor Inn
87 Delany Ave, Bright VIC 3741, Australia
4.5
-------------------
BRIGHT
-------------------
5
-------------------
-------------------
Bright Colonial Motel
56 Gavan St, Bright VIC 3741, Australia
4.6
-------------------
BRIGHT
-------------------
6
-------------------
-------------------
Ovens View
1 Riverside Ave, Bright VIC 3741, Australia
4
-------------------
BRIGHT
-------------

----------------------
-------------------
WONTHAGGI
-------------------
1
-------------------
-------------------
Wonthaggi Motel
40-44 McKenzie St, Wonthaggi VIC 3995, Australia
4.3
-------------------
WONTHAGGI
-------------------
2
-------------------
-------------------
Kilcunda Ocean View Motel
3529 Bass Hwy, Kilcunda VIC 3995, Australia
3.9
-------------------
WONTHAGGI
-------------------
3
-------------------
-------------------
Motel on A'Beckett
24 Abeckett St, Inverloch VIC 3996, Australia
4.8
-------------------
WONTHAGGI
-------------------
4
-------------------
-------------------
Inverloch Motel
39 Powlett St, Inverloch VIC 3996, Australia
4.3
-------------------
WONTHAGGI
-------------------
5
-------------------
-------------------
Inverloch Central Motor Inn
32-34 Abeckett St, Inverloch VIC 3996, Australia
4.4
-------------------
WONTHAGGI
-------------------
6
-------------------
-------------------
Inverloch Cabins
18 Williams St, Inverloch VIC 3996, Australia
4.4


----------------------
-------------------
CAMBERWELL
-------------------
1
-------------------
-------------------
Motel Maroondah
768 Whitehorse Rd, Mont Albert VIC 3127, Australia
2.5
-------------------
CAMBERWELL
-------------------
2
-------------------
-------------------
Best Western Plus The Tudor
1101 Whitehorse Rd, Box Hill VIC 3128, Australia
3.6
-------------------
CAMBERWELL
-------------------
3
-------------------
-------------------
Evancourt Motel
1015/1019 Princes Hwy Service Rd, Malvern East VIC 3145, Australia
3
-------------------
CAMBERWELL
-------------------
4
-------------------
-------------------
Lamplighter Motel
1440 Dandenong Rd, Oakleigh VIC 3166, Australia
3
-------------------
CAMBERWELL
-------------------
5
-------------------
-------------------
Carnegie Motor Inn
1102 Dandenong Rd, Carnegie VIC 3163, Australia
3
-------------------
CAMBERWELL
-------------------
6
-------------------
-------------------
Burwood East Motel
355 Blackburn Road, Melbou

----------------------
-------------------
NARRE WARREN
-------------------
1
-------------------
-------------------
Casey on Princes Motel
218 Princes Hwy, Hallam VIC 3803, Australia
3.6
-------------------
NARRE WARREN
-------------------
2
-------------------
-------------------
Beaconsfield Lodge Motel
3-7 Souter St, Beaconsfield VIC 3807, Australia
2.4
-------------------
NARRE WARREN
-------------------
3
-------------------
-------------------
Cranbourne Motor Inn
1449 S Gippsland Hwy, Cranbourne VIC 3977, Australia
3.1
-------------------
NARRE WARREN
-------------------
4
-------------------
-------------------
Prince Mark Motor Inn
4 Power Rd, Doveton VIC 3177, Australia
3.7
-------------------
NARRE WARREN
-------------------
5
-------------------
-------------------
Dandenong Motel
147 Princes Hwy, Dandenong VIC 3175, Australia
3.5
-------------------
NARRE WARREN
-------------------
6
-------------------
-------------------
Sandown Heritage Motor Inn
433 Princes Hwy, Nobl

----------------------
-------------------
CAMPERDOWN
-------------------
1
-------------------
-------------------
Camperdown Cascade Motel
311 Manifold St, Camperdown VIC 3260, Australia
4.1
-------------------
CAMPERDOWN
-------------------
2
-------------------
-------------------
Manifold Motor Inn
295 Manifold St, Camperdown VIC 3260, Australia
4.6
-------------------
CAMPERDOWN
-------------------
3
-------------------
-------------------
Amble Inn Motel
71 Manifold St, Camperdown VIC 3260, Australia
3.2
-------------------
CAMPERDOWN
-------------------
4
-------------------
-------------------
The Camperdown Mill
3-5 Curdie St, Camperdown VIC 3260, Australia
4.6
-------------------
CAMPERDOWN
-------------------
5
-------------------
-------------------
Lakes and Craters Holiday Park
220 Park Rd, Camperdown VIC 3260, Australia
4.1
-------------------
CAMPERDOWN
-------------------
6
-------------------
-------------------
Camperdown Accommodation: Edwardia short-stay
194a Mani

...

----------------------
-------------------
LEONGATHA
-------------------
1
-------------------
-------------------
Leongatha Motel
18 Turner St, Leongatha VIC 3953, Australia
2.6
-------------------
LEONGATHA
-------------------
2
-------------------
-------------------
Opal Motel
155 S Gippsland Hwy, Leongatha VIC 3953, Australia
3.4
-------------------
LEONGATHA
-------------------
3
-------------------
-------------------
Coal Creek Motel
8444 S Gippsland Hwy, Korumburra VIC 3950, Australia
3.6
-------------------
LEONGATHA
-------------------
4
-------------------
-------------------
Meeniyan Motel
119-129 Whitelaw St, Meeniyan VIC 3956, Australia
3.7
-------------------
LEONGATHA
-------------------
5
-------------------
-------------------
Inverloch Central Motor Inn
32-34 Abeckett St, Inverloch VIC 3996, Australia
4.4
-------------------
LEONGATHA
-------------------
6
-------------------
-------------------
1st T Motel
18 Galvins Rd, Mirboo North VIC 3871, Australia
4.5
-------

----------------------
-------------------
SWAN HILL
-------------------
1
-------------------
-------------------
Jane Eliza Motor Inn
263 Campbell St, Swan Hill VIC 3585, Australia
4.3
-------------------
SWAN HILL
-------------------
2
-------------------
-------------------
Paddle Steamer Motel
5563 Murray Valley Hwy, Swan Hill VIC 3585, Australia
4.4
-------------------
SWAN HILL
-------------------
3
-------------------
-------------------
Best Western Burke and Wills Motor Inn
370 Campbell St, Swan Hill VIC 3585, Australia
4.5
-------------------
SWAN HILL
-------------------
4
-------------------
-------------------
Pioneer Station Motor Inn
421 Campbell St, Swan Hill VIC 3585, Australia
3.6
-------------------
SWAN HILL
-------------------
5
-------------------
-------------------
Australian Settlers Motor Inn
354 Campbell St, Swan Hill VIC 3585, Australia
4.4
-------------------
SWAN HILL
-------------------
6
-------------------
-------------------
Lady Augusta Swan Hill
375

----------------------
-------------------
SALE
-------------------
1
-------------------
-------------------
ibis Styles Sale
25-41 Princes Hwy, Sale VIC 3850, Australia
3.6
-------------------
SALE
-------------------
2
-------------------
-------------------
Comfort Inn & Suites King Avenue
20-26 Princes Hwy, Sale VIC 3850, Australia
4.3
-------------------
SALE
-------------------
3
-------------------
-------------------
Motel24seven.com.au & Apartments
386 York St, Sale VIC 3850, Australia
4.4
-------------------
SALE
-------------------
4
-------------------
-------------------
THE WURRUK MOTEL
46-54 Wurruk Way, Wurruk VIC 3850, Australia
3.8
-------------------
SALE
-------------------
5
-------------------
-------------------
Rivers Apartments & Motel Sale Gippsland
1/1/15 Princes Hwy, Wurruk VIC 3850, Australia
4.1
-------------------
SALE
-------------------
6
-------------------
-------------------
The Matador Motel
Princes Hwy &, Raymond St, Sale VIC 3850, Australia
4.5
--

----------------------
-------------------
WODONGA
-------------------
1
-------------------
-------------------
Motel Wellington Wodonga
46 High St, Wodonga VIC 3690, Australia
4
-------------------
WODONGA
-------------------
2
-------------------
-------------------
Warrina Motor Inn
31 High St, Wodonga VIC 3690, Australia
4.1
-------------------
WODONGA
-------------------
3
-------------------
-------------------
MOTEL 24
13 High St, Wodonga VIC 3690, Australia
4.5
-------------------
WODONGA
-------------------
4
-------------------
-------------------
Stagecoach Motel Wodonga
188 Melbourne Rd, Wodonga VIC 3690, Australia
4.6
-------------------
WODONGA
-------------------
5
-------------------
-------------------
Quest Wodonga
46 Reid St, Wodonga VIC 3690, Australia
4.3
-------------------
WODONGA
-------------------
6
-------------------
-------------------
Twin City Motor Inn
166 Melbourne Rd, Wodonga VIC 3690, Australia
3.8
-------------------
WODONGA
-------------------
7
--

----------------------
-------------------
WARRACKNABEAL
-------------------
1
-------------------
-------------------
Country Roads Motor Inn
197 Henty Hwy, Warracknabeal VIC 3393, Australia
4
-------------------
WARRACKNABEAL
-------------------
2
-------------------
-------------------
Warrack Motel
2 Lyle St, Warracknabeal VIC 3393, Australia
3.8


# Create and clean df from accommodation data gathered 

In [74]:
print("Total returned Accommodation Options: "+ str(len(name_stay)))

Total returned Accommodation Options: 1204


In [85]:
#create a df from acommodation
accommodation_df = pd.DataFrame(name_stay)
accommodation_df["accommodation_address"]= address_stay
accommodation_df["accommodation_rating"]= rating_stay
accommodation_df["accommodation_city"]= city_search_stay
# Rename column 0 accommodation_df
accommodation_df = accommodation_df.rename(columns={0: "accommodation_name"})
accommodation_df

,accommodation_name,accommodation_address,accommodation_rating,accommodation_city
0,John Bright Motor Inn,"10-12 Wood St, Bright VIC 3741, Australia",4.3,BRIGHT
1,Riverbank Park Motel,"69 Gavan St, Bright VIC 3741, Australia",4.1,BRIGHT
2,High Country Motor Inn,"13-17 Great Alpine Rd, Bright VIC 3741, Australia",3.9,BRIGHT
3,Bright Avenue Motor Inn,"87 Delany Ave, Bright VIC 3741, Australia",4.5,BRIGHT
4,Bright Colonial Motel,"56 Gavan St, Bright VIC 3741, Australia",4.6,BRIGHT
...,...,...,...,...
1199,Mountain Lodge Dandenong Ranges Accommodation,"1354 Mount Dandenong Tourist Rd, Mount Dandeno...",5.0,LILYDALE
1200,Cottages Monreale,"81 The Crescent, Sassafras VIC 3787, Australia",3.7,LILYDALE
1201,Maroondah 3 Bedroom house,"1/8 Browning St, Kilsyth VIC 3137, Australia",4.5,LILYDALE
1202,Country Roads Motor Inn,"197 Henty Hwy, Warracknabeal VIC 3393, Australia",4.0,WARRACKNABEAL


In [86]:
#drop duplicates in accommodation_df
accommodation_xdbl_df=accommodation_df.drop_duplicates(subset=['accommodation_name', 'accommodation_city'],keep='last')
accommodation_xdbl_df

,accommodation_name,accommodation_address,accommodation_rating,accommodation_city
0,John Bright Motor Inn,"10-12 Wood St, Bright VIC 3741, Australia",4.3,BRIGHT
1,Riverbank Park Motel,"69 Gavan St, Bright VIC 3741, Australia",4.1,BRIGHT
2,High Country Motor Inn,"13-17 Great Alpine Rd, Bright VIC 3741, Australia",3.9,BRIGHT
3,Bright Avenue Motor Inn,"87 Delany Ave, Bright VIC 3741, Australia",4.5,BRIGHT
4,Bright Colonial Motel,"56 Gavan St, Bright VIC 3741, Australia",4.6,BRIGHT
...,...,...,...,...
1199,Mountain Lodge Dandenong Ranges Accommodation,"1354 Mount Dandenong Tourist Rd, Mount Dandeno...",5.0,LILYDALE
1200,Cottages Monreale,"81 The Crescent, Sassafras VIC 3787, Australia",3.7,LILYDALE
1201,Maroondah 3 Bedroom house,"1/8 Browning St, Kilsyth VIC 3137, Australia",4.5,LILYDALE
1202,Country Roads Motor Inn,"197 Henty Hwy, Warracknabeal VIC 3393, Australia",4.0,WARRACKNABEAL


In [87]:
# save csv of accommodation
accommodation_xdbl_df.to_csv("Cleaned_Data/lga_accommodation.csv", encoding="utf-8", index=False)

# Get average high temps for LGA cities

In [176]:
#loop through the cities to scrape climate history data for each city
weather= []
city_weather = []

for city in cities:
    #set base url
    url = f'https://www.eldersweather.com.au/climate-history/vic/{city}'
    # Retrieve page with the requests module
    response = requests.get(url)
    # Create a Beautiful Soup object
    soup = bs(response.text, 'html.parser')
    # set to find children in html table
    tables = soup.findChildren('table')
    my_table = tables[0]
    rows = my_table.findChildren(['th', 'tr'])
    for row in rows:
        cells = row.findChildren('td')       
        for cell in cells:
            value = cell.string
            value = value.replace('\n','')
            weather.append(value)
            city_weather.append(city)
    

In [180]:
print(len(weather))

4108


In [181]:
obs_needed = 4108/79
obs_needed

52.0

In [194]:
obs = ['jan_max','feb_max','mar_max','apr_max','may_max','jun_max','jul_max','aug_max','sep_max',
       'oct_max','nov_max','dec_max','ann_max', 
       'jan_min','feb_min','mar_min','apr_min','may_min','jun_min','jul_min','aug_min','sep_min',
       'oct_min','nov_min','dec_min','ann_min',
       'jan_mm','feb_mm','mar_mm','apr_mm','may_mm','jun_mm','jul_mm','aug_mm','sep_mm','oct_mm',
       'nov_mm','dec_mm','ann_mm',
       'jan_rain_days','feb_rain_days','mar_rain_days','apr_rain_days','may_rain_days',
       'jun_rain_days','jul_rain_days','aug_rain_days','sep_rain_days','oct_rain_days',
       'nov_rain_days','dec_rain_days','ann_rain_days']

In [199]:
# loop to create enough sets of obs to match weather data to make df
obs_list=[]
counter = 0
for x in range(79):
    for ob in obs:
        obs_list.append(ob)
    
    
       

In [200]:
print(len(obs_list))

4108


In [203]:
#create a weather obs df

#create a df from weather
av_weather_df = pd.DataFrame(obs_list)
av_weather_df['averages']= weather
av_weather_df['city']= city_weather

# Rename column 0 accommodation_df
av_weather_df = av_weather_df.rename(columns={0: "long-term_avg_obs"})
av_weather_df

,long-term_avg_obs,averages,city
0,jan_max,29.2,BRIGHT
1,feb_max,29.1,BRIGHT
2,mar_max,25.8,BRIGHT
3,apr_max,20.8,BRIGHT
4,may_max,16.0,BRIGHT
...,...,...,...
4103,sep_rain_days,12.0,WARRACKNABEAL
4104,oct_rain_days,8.5,WARRACKNABEAL
4105,nov_rain_days,7.0,WARRACKNABEAL
4106,dec_rain_days,5.7,WARRACKNABEAL


In [204]:
# save csv of weather obs
av_weather_df.to_csv("Cleaned_Data/weather.csv", encoding="utf-8", index=False)